# This script will convert the FIRE hdf5 files into a JSON data file that can be read into WebGLonFIRE 


### Python imports

In [1]:
import numpy as np
import pandas as pd
import h5py,sys,getopt,os,random

### The class that will allow you to read in files within a directory , create the dictionary, and write out the json file

#### Everything within "init" are defaults that can be modified by the user (see below)

In [2]:
class FIREreader(object):
    """
    #These are the defaults that can be redefined by the user at runtime

        #directory that contains all the hdf5 data files
        self.directory = './' 
        
        #amount to decimate the data (==1 means no decimates, >1 factor by which to reduce the amount of data)
        self.decimate = 1. 

        #particles to return
        self.returnParts = ['PartType0', 'PartType1', 'PartType2', 'PartType4']
 
        #set names for the particle sets (note: these will be used in the UI of WebGLonFIRE; 4 characters or less is best)
        self.swaps = {'PartType0':'Gas', 
                      'PartType1':'HRDM', 
                      'PartType2':'LRDM', 
                      'PartType4':'Stars' }
        
        #keys from the hd5 file to include in the JSON file for WebGLonFIRE (must at least include Coordinates)
        self.returnKeys = {'Gas': ['Coordinates'],  
                           'HRDM': ['Coordinates'],  
                           'LRDM': ['Coordinates'],  
                           'Stars': ['Coordinates'] }         
        
        
        #set the default colors = rgba.  The alpha value here becomes a multiplier if weights are provided. 
        #NOTE: this must use the names from self.swaps.
        self.colors = {'Gas': [1., 0., 0., 0.1],  
                       'HRDM': [1., 1., 0., 0.1],  
                       'LRDM': [1., 1., 0., 0.1],  
                       'Stars': [0., 0., 1., 0.1] } 

        #set the weight of the particles (to define the alpha value). This is a function that will calculate the weights
        #NOTE: this must use the names from self.swaps.
        self.weightFunction = {'Gas': None, 
                      'HRDM': None, 
                      'LRDM': None, 
                      'Stars': None}
        
        #set the radii of the particles. This is a function that will calculate the radii
        #NOTE: this must use the names from self.swaps.
        self.radiusFunction = {'Gas': None, 
                      'HRDM': None, 
                      'LRDM': None, 
                      'Stars': None}
        
        #set the default point size multiplier 
        #NOTE: this must use the names from self.swaps.
        self.sizeMult = {'Gas':0.1, 
                      'HRDM':0.1, 
                      'LRDM':0.1, 
                      'Stars':0.1 }
                      
        #set the number of points to plot during each draw (larger numbers will make the visualization run more slowly)
        #NOTE: this must use the names from self.swaps.
        self.nMaxPlot = {'Gas':1e4, 
                      'HRDM':1e4, 
                      'LRDM':1e4, 
                      'Stars':1e4 }
                      
        #decide whether you want to use the key from returnKeys as a filter item in the UI
        #NOTE: this must use the names from self.swaps.
        #NOTE: each of these must be the same length as self.returnKeys
        self.addFilter = {'Gas': [False],  
                           'HRDM': [False],  
                           'LRDM': [False],  
                           'Stars': [False] }        
  
        #should we use the log of these values?  
        #NOTE: this must use the names from self.swaps.
        #NOTE: this must be the same length as self.returnKeys
        self.dolog = {'Gas': [False],  
                       'HRDM': [False],  
                       'LRDM': [False],  
                       'Stars': [False] }          

        #should we use the magnitude of these values?   
        #NOTE: this must use the names from self.swaps.
        #NOTE: this must be the same length as self.returnKeys
        #NOTE: setting any of these to true will significantly slow down the file creation
        self.domag = {'Gas': [False],  
                       'HRDM': [False],  
                       'LRDM': [False],  
                       'Stars': [False] }  
 
        #should we plot using Alex Gurvich's radial profile fit to the SPH particles (==1), or a simple symmetric radial profile?   
        #NOTE: this must use the names from self.swaps.
        #NOTE: this must be the same length as self.returnKeys
        self.doSPHrad = {'Gas': [1],
                       'HRDM': [0],  
                       'LRDM': [0],  
                       'Stars': [0] }  
        
        #the name of the JSON file
        self.JSONfname = 'FIREdata.json'
        
        #in case you want to print the available keys to the screen
        self.showkeys = False

    """


    def __init__(self, *args,**kwargs):
##################################################        
#defaults that can be modified

        #directory that contains all the hdf5 data files
        self.directory = './' 
        
        #amount to decimate the data (==1 means no decimates, >1 factor by which to reduce the amount of data)
        self.decimate = 1. 

        #particles to return
        self.returnParts = ['PartType0', 'PartType1', 'PartType2', 'PartType4']
 
        #set names for the particle sets (note: these will be used in the UI of WebGLonFIRE; 4 characters or less is best)
        self.swaps = {'PartType0':'Gas', 
                      'PartType1':'HRDM', 
                      'PartType2':'LRDM', 
                      'PartType4':'Stars' }
        
        #keys from the hd5 file to include in the JSON file for WebGLonFIRE (must at least include Coordinates)
        self.returnKeys = {'Gas': ['Coordinates'],  
                           'HRDM': ['Coordinates'],  
                           'LRDM': ['Coordinates'],  
                           'Stars': ['Coordinates'] }         
        
        
        #set the default colors = rgba.  The alpha value here becomes a multiplier if weights are provided. 
        #NOTE: this must use the names from self.swaps.
        self.colors = {'Gas': [1., 0., 0., 0.1],  
                       'HRDM': [1., 1., 0., 0.1],  
                       'LRDM': [1., 1., 0., 0.1],  
                       'Stars': [0., 0., 1., 0.1] } 

        #set the weight of the particles (to define the alpha value). This is a function that will calculate the weights
        #NOTE: this must use the names from self.swaps.
        self.weightFunction = {'Gas': None, 
                      'HRDM': None, 
                      'LRDM': None, 
                      'Stars': None}
        
        #set the radii of the particles. This is a function that will calculate the radii
        #NOTE: this must use the names from self.swaps.
        self.radiusFunction = {'Gas': None, 
                      'HRDM': None, 
                      'LRDM': None, 
                      'Stars': None}
        
        #set the default point size multiplier 
        #NOTE: this must use the names from self.swaps.
        self.sizeMult = {'Gas':0.1, 
                      'HRDM':0.1, 
                      'LRDM':0.1, 
                      'Stars':0.1 }

        #set the number of points to plot during each draw (larger numbers will make the visualization run more slowly)
        #NOTE: this must use the names from self.swaps.
        self.nMaxPlot = {'Gas':1e4, 
                      'HRDM':1e4, 
                      'LRDM':1e4, 
                      'Stars':1e4 }
        
        #decide whether you want to use the key from returnKeys as a filter item in the UI
        #NOTE: this must use the names from self.swaps.
        #NOTE: each of these must be the same length as self.returnKeys
        self.addFilter = {'Gas': [False],  
                           'HRDM': [False],  
                           'LRDM': [False],  
                           'Stars': [False] }        
  
        #should we use the log of these values?  
        #NOTE: this must use the names from self.swaps.
        #NOTE: this must be the same length as self.returnKeys
        self.dolog = {'Gas': [False],  
                       'HRDM': [False],  
                       'LRDM': [False],  
                       'Stars': [False] }          

        #should we use the magnitude of these values?   
        #NOTE: this must use the names from self.swaps.
        #NOTE: this must be the same length as self.returnKeys
        #NOTE: setting any of these to true will significantly slow down the file creation
        self.domag = {'Gas': [False],  
                       'HRDM': [False],  
                       'LRDM': [False],  
                       'Stars': [False] }  
 
        #should we plot using Alex Gurvich's radial profile fit to the SPH particles (==1), or a simple symmetric radial profile?   
        #NOTE: this must use the names from self.swaps.
        #NOTE: this must be the same length as self.returnKeys
        self.doSPHrad = {'Gas': [1],
                       'HRDM': [0],  
                       'LRDM': [0],  
                       'Stars': [0] }  
        
        #the name of the JSON file
        self.JSONfname = 'FIREdata'
        
        #in case you want to print the available keys to the screen
        self.showkeys = False
        
################################################## 
#don't modify these

        #the data for the JSON file
        self.partsDict = dict()
        
        #keys that shouldn't be shuffled or decimated
        self.nodecimate = ['color','sizeMult','filterKeys','doSPHrad', 'nMaxPlot']
        
        #keys for filtering (will be defined below)
        self.filterKeys = {}
        
################################################## 
################################################## 
################################################## 
        
    #used self.swaps to swap the dictionary keys
    def swapnames(self, pin):
        return self.swaps[pin]

    #adds an array to the dict for a given particle set and data file 
    def addtodict(self, d, snap, part, dkey, sendlog, sendmag, usekey = None, mfac = 1.):
        if (usekey == None):
            ukey = dkey
        else:
            ukey = usekey
            
        vals = snap[part + '/' + dkey][...] * mfac      
        if (sendlog):
            ukey = "log10"+ukey
            vals = np.log10(vals)
        if (sendmag):  
            #print "calculating magnitude for ", ukey
            ukey = "mag"+ukey
            vals = [np.linalg.norm(v) for v in vals]
         
        if ukey in d[part].keys():
            d[part][ukey] = np.append(vals, d[part][ukey], axis=0)
        else:
            d[part][ukey] = vals

                                   
    #populate the dict
    def populate_dict(self):
        for fname in os.listdir(self.directory):
            print(fname)
            with h5py.File(self.directory + '/' + fname,'r') as snap:
                parts = snap.keys()[1:]
                for p in parts:
                    if p in self.returnParts:
                        if p not in self.partsDict:
                            self.partsDict[p] = dict()
                        vals = snap[p].keys()
                        pp = self.swapnames(p)
                        #This shows the available keys
                        if (self.showkeys):
                            print(p,pp, vals)
                        if (self.radiusFunction[pp] != None):
                            self.radiusFunction[pp](self, self.partsDict, snap, p)
                        if (self.weightFunction[pp] != None):
                            self.weightFunction[pp](self, self.partsDict, snap, p)
                        for i,k in enumerate(self.returnKeys[pp]):
                            if (k in vals):
                                self.addtodict(self.partsDict, snap, p, k, self.dolog[pp][i], self.domag[pp][i])

                            
        #swap the names to something more useful
        #and add on the colors and point size defaults
        #also calculate the magnitude where necessary
        for p in self.partsDict.keys():
            pp = self.swapnames(p)
            self.partsDict[pp] = self.partsDict.pop(p)
            self.partsDict[pp]['color'] = self.colors[pp]
            self.partsDict[pp]['sizeMult'] = self.sizeMult[pp]
            self.partsDict[pp]['filterKeys'] = self.filterKeys[pp]
            self.partsDict[pp]['doSPHrad'] = self.doSPHrad[pp]
            self.partsDict[pp]['nMaxPlot'] = self.nMaxPlot[pp]

                    
        #should we decimate the data? (NOTE: even if decimate = 1, it is wise to shuffle the data so it doesn't display in blocks)
        if (self.decimate > 0): 
            if (self.decimate > 1):
                print("decimating and shuffling ...")
            else:
                print("shuffling ... ")
            parts = self.partsDict.keys()
            for p in parts:
                N = int(len(self.partsDict[p][self.returnKeys[p][0]]))
                indices = np.arange(N )
                dindices = np.random.choice(indices, size = int(round(N/self.decimate)))
                for k in self.partsDict[p].keys():
                    if (k not in self.nodecimate):
                        self.partsDict[p][k] = self.partsDict[p][k][dindices]

    #create the JSON file, and then add the name of the variable (parts) that we want in WebGLonFIRE
    def createJSON(self):
        print("writing JSON files ...")
        #first create the dict of file names and write that to a JSON file
        filenames = dict()
        for p in self.partsDict:
            filenames[p] = self.JSONfname+p+'.json'
            print(filenames[p])
            pd.Series(self.partsDict[p]).to_json(filenames[p], orient='index') 
        pd.Series(filenames).to_json('filenames.json', orient='index') 
        
    
    def defineFilterKeys(self):
        for p in names:
            self.filterKeys[p] = []
            j = 0
            for i,k in enumerate(self.returnKeys[p]):
                if (self.addFilter[p][i]):
                    self.filterKeys[p].append(k)
                    if (self.dolog[p][i]):
                        self.filterKeys[p][j] = 'log10' + self.filterKeys[p][j]
                    if self.domag[p][i]:
                        self.filterKeys[p][j] = 'mag' + self.filterKeys[p][j]
                    j += 1
        #print "filters = ", self.filterKeys
        


    def run(self):
        self.defineFilterKeys()
        self.populate_dict()
        self.createJSON()
        print("done")

# Set the defaults and create the JSON

In [3]:
#a user defined function to calculate the radius, usekey must be partRadius
def calcRadius(self, data, snap, p):
    if ('SmoothingLength' in snap[p]):
        #print("calculating SmoothinLength radius", p)
        self.addtodict(data, snap, p, 'SmoothingLength', False, False, usekey="partRadius")

#a user defined function to calculate the weights, usekey must be partWeight
def calcWeight(self, data, snap, p):
    if ('Density' in snap[p]):
        #print("calculating Density weight", p)
        self.addtodict(data, snap, p, 'Density', False, False, usekey="partWeight", mfac = 100000.)


reader = FIREreader()

#modify the defaults here
#reader.directory = "/Users/ageller/Visualizations/Firefly/snapdir_440"
reader.directory = "/Users/ageller/Visualizations/Firefly/snapdir_050"
reader.decimate = 1. 

reader.returnParts = ['PartType0', 'PartType4']
names = ['Gas', 'Stars']

#names = ['Gas', 'HRDM', 'LRDM', 'Stars']

for p in names:
    reader.returnKeys[p] = ['Coordinates', 'Density','Velocities']
    reader.addFilter[p] = [False, True, False]
    reader.dolog[p] = [False, True, False]
    reader.domag[p] = [False, False, False]#NOTE: calculating the magnitudes takes time. (I calculate magnitude of velocity, if Velocities is supplied, in the web app)..

    reader.radiusFunction[p] = calcRadius
    reader.sizeMult[p] = 0.1
    reader.nMaxPlot[p] = 1e10

    reader.weightFunction[p] = calcWeight
    
    
reader.colors = {'Gas': [1., 0., 0., 1.],  
           'HRDM': [1., 1., 0., 0.1],  
           'LRDM': [1., 1., 0., 0.1],  
           'Stars': [0., 0., 1., 0.1] } 

        
reader.showkeys = False

#make the file
reader.run()





snapshot_050.hdf5
shuffling ... 
writing JSON files ...
FIREdataGas.json
FIREdataStars.json
done
